In [5]:
import pandas as pd
import os
import numpy as np

In [7]:
path = 'data/Ridership Data Files'

dl = []
for file in os.listdir(path):
    if file[-4:] == 'xlsx':
        flist = []
        f = path + '/' + file
        sheet_list = pd.read_excel(f, sheet_name=None, header=None)
        for name, sheet in sheet_list.items():
            # Exclude Totals sheets and empty sheets
            if 'Totals' not in name and 'Sheet' not in name:
                #print(file, name)

                # Remove empty rows and summary rows
                sheet.dropna(how='all', inplace=True)
                sheet = sheet[~sheet.iloc[:,1].str.contains('Week', na=False)]

                # First row has grouped titles, so fill right empty cells
                first_row = sheet.loc[[0]]
                first_row = first_row.ffill(axis=1)
                sheet = sheet.iloc[1:]
                sheet = pd.concat([first_row, sheet])
                sheet.reset_index(inplace=True)

                # Combine first and second rows for column titles
                tuple_list = zip(sheet.values.tolist()[0], sheet.values.tolist()[1])

                l = []
                for t in tuple_list:
                    pair = str(t[0]) + ' : ' + str(t[1])
                    l.append(pair)

                sheet.columns = l
                sheet = sheet.reset_index(drop=True)

                # Clean up unnecessary columns and rows
                sheet = sheet.iloc[2:, 1:]
                sheet = sheet[sheet.iloc[:,1].notna()]
                sheet = sheet[sheet.columns[~sheet.columns.str.contains('Total')]]

                sheet = sheet.rename(columns={sheet.columns[0]: 'Day',
                                              sheet.columns[1]: 'Date'})

                sheet.fillna(0, inplace=True)
                sheet['Company'] = name

                # Unpivot columns
                sheet = pd.melt(sheet, id_vars=['Day', 'Date', 'Company'],
                                var_name='Route', value_name='Rides')

                # Clean up
                sheet = sheet[sheet['Day'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])]
                #sheet['Filename'] = file
                sheet['Rides'] = pd.to_numeric(sheet['Rides'], errors='coerce')
                sheet['Rides'].fillna(0, inplace=True)
                sheet['Date'] = pd.to_datetime(sheet['Date'], errors='coerce')
                sheet['Day'] = sheet['Date'].dt.day_name()
                #sheet = sheet[sheet['Rides'].astype(int) >= 0]
                sheet = sheet[sheet['Date'] >= '2013-01-01']
                flist.append(sheet)

        df = pd.concat(flist)
        dl.append(df)

f = pd.concat(dl)

In [8]:
# Remove totals columns that didn't have "Total" in the name
f = f[~f['Route'].str.endswith('nan')]

# Split Route into Route and Landing columns
f[['S1', 'S2']] = f['Route'].str.split(' : ', expand=True)

# Some of them are backwards
f['Landing'] = np.where(f['Company'].isin(['Baseball', 'NYC Ferry']), f['S2'], f['S1'])
f['Route'] = np.where(f['Company'].isin(['Baseball', 'NYC Ferry']), f['S1'], f['S2'])

# Assign Weekend
f['Weekend'] = np.where(f['Day'].isin(['Saturday', 'Sunday']), 'Weekend', 'Weekday')

f.drop(['S1', 'S2'], axis=1, inplace=True)

# Some spreadsheets had dates from the prior month hidden at the top (and had zero values)
# this is to get rid of those extra rows
f = f.groupby(['Day', 'Date', 'Company', 'Route', 'Landing', 'Weekend']).sum().reset_index()

f.to_csv('data/' + 'final.csv')

In [9]:
f

,Day,Date,Company,Route,Landing,Weekend,Rides
0,Friday,2013-01-04,Billy Bey,East River,Atlantic Ave,Weekday,0.0
1,Friday,2013-01-04,Billy Bey,East River,East 34th Street,Weekday,313.0
2,Friday,2013-01-04,Billy Bey,East River,Fulton Ferry,Weekday,268.0
3,Friday,2013-01-04,Billy Bey,East River,Governors Island,Weekday,0.0
4,Friday,2013-01-04,Billy Bey,East River,Greenpoint,Weekday,181.0
...,...,...,...,...,...,...,...
141089,Wednesday,2020-12-30,SeaStreak,Atlantic Highlands,East 34th Street,Weekday,0.0
141090,Wednesday,2020-12-30,SeaStreak,Highlands,BMB Slip #5,Weekday,202.0
141091,Wednesday,2020-12-30,SeaStreak,Highlands,East 34th Street,Weekday,206.0
141092,Wednesday,2020-12-30,SeaStreak,Highlands,Pier 11,Weekday,0.0
